<a href="https://colab.research.google.com/github/rkdune/roadtojepa/blob/main/autoenconder_emadecoder_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install allensdk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.0/336.0 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 13.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import seaborn as sns

In [2]:
# Download the data
cache = BrainObservatoryCache(manifest_file='boc_manifest.json')

expt_id = 649409874
data_set = cache.get_ophys_experiment_data(expt_id)
print(f"Using experiment ID: {expt_id}")

# Get experiments and find one with available data
# experiments = cache.get_ophys_experiments()
# data_set = None
# for experiment in experiments:
#     expt_id = experiment['id']
#     try:
#         data_set = cache.get_ophys_experiment_data(expt_id)
#         # If we get here, we've found a valid dataset
#         print(f"Using experiment ID: {expt_id}")
#         break
#     except Exception as e:
#         print(f"Skipping experiment {expt_id}: {str(e)}")

# if data_set is None:
#     raise Exception("No valid experiments found with available data.")

# Get the fluorescence traces
dff_traces = data_set.get_dff_traces()
neuron_traces = dff_traces[1]

# Normalize the data
neuron_traces = (neuron_traces - np.min(neuron_traces)) / (np.max(neuron_traces) - np.min(neuron_traces))

# Reshape the data for the autoencoder
X = neuron_traces.T
X = X.reshape((X.shape[0], 1, X.shape[1]))  # (samples, channels, time_steps)

2024-07-09 05:32:33,531 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/650079447
INFO:allensdk.api.api.retrieve_file_over_http:Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/650079447


Using experiment ID: 649409874


In [3]:
print(f"Data shape: {X.shape}")

Data shape: (118996, 1, 26)


note for experiment id: 649409874 works :)

In [4]:
# Split the data
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]

# Convert to PyTorch tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

# Create DataLoaders
train_dataset = TensorDataset(X_train, X_train)
test_dataset = TensorDataset(X_test, X_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [151]:
# Assuming X has shape (118996, 1, 26)
# Let's reshape it to (118996, 26) for easier plotting
data = X.squeeze(axis=1)

# # 1. Time series plot of a few neurons
# plt.figure(figsize=(15, 5))
# for i in range(5):  # Plot first 5 neurons
#     plt.plot(data[:1000, i], label=f'Neuron {i+1}')
# plt.title('Activity of 5 Neurons over Time')
# plt.xlabel('Time Steps')
# plt.ylabel('Normalized Activity')
# plt.legend()
# plt.show()

# # 2. Heatmap of neuron activities
# plt.figure(figsize=(12, 8))
# sns.heatmap(data[:1000, :].T, cmap='viridis')
# plt.title('Heatmap of Neuron Activities')
# plt.xlabel('Time Steps')
# plt.ylabel('Neurons')
# plt.show()

# # 3. Distribution of activities for each neuron
# plt.figure(figsize=(15, 5))
# plt.boxplot([data[:, i] for i in range(data.shape[1])])
# plt.title('Distribution of Activities for Each Neuron')
# plt.xlabel('Neuron')
# plt.ylabel('Normalized Activity')
# plt.show()

# # 4. Correlation matrix between neurons
# correlation_matrix = np.corrcoef(data.T)
# plt.figure(figsize=(10, 8))
# sns.heatmap(correlation_matrix, cmap='coolwarm', center=0)
# plt.title('Correlation Matrix Between Neurons')
# plt.show()

# # 5. Average activity over time
# avg_activity = data.mean(axis=1)
# plt.figure(figsize=(15, 5))
# plt.plot(avg_activity[:1000])
# plt.title('Average Neuron Activity Over Time')
# plt.xlabel('Time Steps')
# plt.ylabel('Average Normalized Activity')
# plt.show()

In [76]:
print("Input data shape:", X.shape)

Input data shape: (118996, 1, 26)


In [146]:
class Autoencoder(nn.Module):
    def __init__(self, input_size, hidden_size, ema_decay=0.99999):
        super(Autoencoder, self).__init__()
        self.input_size = input_size
        self.ema_decay = ema_decay

        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.LeakyReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.LeakyReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.LeakyReLU(),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.LeakyReLU()
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(hidden_size // 2, hidden_size),
            nn.LeakyReLU(),
            nn.Linear(hidden_size, input_size),
            nn.LeakyReLU()
        )

        # Initialize EMA weights for decoder
        self.ema_decoder = self.create_ema_copy(self.decoder)

    def create_ema_copy(self, model):
        ema_model = type(model)(model)
        for ema_param, param in zip(ema_model.parameters(), model.parameters()):
            ema_param.data.copy_(param.data)
            ema_param.requires_grad_(False)
        return ema_model

    def update_ema_weights(self):
        with torch.no_grad():
            for ema_param, param in zip(self.ema_decoder.parameters(), self.decoder.parameters()):
                ema_param.data.mul_(self.ema_decay).add_(param.data, alpha=1 - self.ema_decay)

    def forward(self, x):
        # Reshape input if necessary
        original_shape = x.shape
        x = x.view(x.size(0), -1)

        # Encode
        encoded = self.encoder(x)

        # Decode using EMA weights
        decoded = self.ema_decoder(encoded)

        # Reshape output back to original shape if necessary
        return decoded.view(original_shape)

In [149]:
# Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# input_size = X.shape[1] * X.shape[2]  # Flatten the input
hidden_size = 32  # You can adjust this

# Initialize the model
input_size = X.shape[2]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Autoencoder(input_size, hidden_size).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
num_epochs = 150
train_losses = []
test_losses = []

In [150]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch_features, _ in train_loader:
        batch_features = batch_features.to(device)
        optimizer.zero_grad()
        outputs = model(batch_features)
        loss = criterion(outputs, batch_features)
        loss.backward()
        optimizer.step()
        model.update_ema_weights()
        train_loss += loss.item()
    train_loss /= len(train_loader)
    train_losses.append(train_loss)

    model.eval()
    test_loss = 0
    with torch.no_grad():
        for batch_features, _ in test_loader:
            batch_features = batch_features.to(device)
            outputs = model(batch_features)
            loss = criterion(outputs, batch_features)
            test_loss += loss.item()
    test_loss /= len(test_loader)
    test_losses.append(test_loss)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.8f}, Test Loss: {test_loss:.8f}')

Epoch [1/150], Train Loss: 0.00895893, Test Loss: 0.00862382
Epoch [2/150], Train Loss: 0.00841103, Test Loss: 0.00786913
Epoch [3/150], Train Loss: 0.00778451, Test Loss: 0.00769222
Epoch [4/150], Train Loss: 0.00762558, Test Loss: 0.00754644
Epoch [5/150], Train Loss: 0.00749712, Test Loss: 0.00740826
Epoch [6/150], Train Loss: 0.00738945, Test Loss: 0.00733320
Epoch [7/150], Train Loss: 0.00734696, Test Loss: 0.00729853
Epoch [8/150], Train Loss: 0.00731707, Test Loss: 0.00726512
Epoch [9/150], Train Loss: 0.00729201, Test Loss: 0.00724210
Epoch [10/150], Train Loss: 0.00726229, Test Loss: 0.00720192
Epoch [11/150], Train Loss: 0.00724314, Test Loss: 0.00719144
Epoch [12/150], Train Loss: 0.00723533, Test Loss: 0.00718035
Epoch [13/150], Train Loss: 0.00722955, Test Loss: 0.00717311
Epoch [14/150], Train Loss: 0.00722468, Test Loss: 0.00716540
Epoch [15/150], Train Loss: 0.00722048, Test Loss: 0.00716569
Epoch [16/150], Train Loss: 0.00721631, Test Loss: 0.00715439
Epoch [17/150], T

KeyboardInterrupt: 

In [ ]:
# Verification of encoder gradients and unique outputs
model.eval()
with torch.no_grad():
    sample1 = next(iter(test_loader))[0][:1].to(device)
    sample2 = next(iter(test_loader))[0][:1].to(device)
    output1 = model(sample1)
    output2 = model(sample2)
    print("Output difference:", torch.abs(output1 - output2).mean().item())

for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"{name} has gradient: {param.grad is not None}")

In [ ]:
# Plot the training history
plt.figure(figsize=(12, 4))
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.show()

# Encode and decode some test traces
model.eval()
with torch.no_grad():
    test_samples = X_test[:5].to(device)
    reconstructed = model(test_samples).cpu().numpy()

In [ ]:
# Select a few samples from the test set
n = 5
test_samples = X_test[:n].to(device)
reconstructed = model(test_samples).cpu().detach()

# Calculate average activity
avg_original = torch.mean(X_test[:, 0, :], dim=0)
avg_reconstructed = torch.mean(reconstructed[:, 0, :], dim=0)

# Display original and reconstructed traces, including average
plt.figure(figsize=(20, 3))  # Slightly wider to accommodate the 6th plot
for i in range(n + 1):  # n+1 to include the average plot
    ax = plt.subplot(1, n + 1, i + 1)

    if i < n:
        # Individual samples
        plt.plot(X_test[i, 0, :].cpu().numpy(), label='Original', alpha=1, color = "sienna")
        plt.plot(reconstructed[i, 0, :].numpy(), label='Reconstructed', alpha=1, linestyle='-', color = "teal")
        plt.title(f"Sample {i+1}")
    else:
        # Average plot
        plt.plot(avg_original.cpu().numpy(), label='Original', alpha=1, color = "sienna")
        plt.plot(avg_reconstructed.numpy(), label='Reconstructed', alpha=1, linestyle='-', color = "teal")
        plt.title("Average")

    plt.ylim(0, 0.6)

    if i == 0:
        plt.ylabel("Normalized\nFluorescence (ΔF/F)")
    else:
        plt.yticks([])

    plt.xlabel("Time Steps")

    if i == n:  # Add legend to the last plot (average)
        plt.legend(loc='upper right', fontsize='small')

plt.tight_layout()
plt.subplots_adjust(top=0.85)
plt.show()